In [1]:
import numpy as np
import pandas as pd
import re

In [20]:
api_info = pd.read_csv("Extracted_Class_Functions_and_Descriptions.csv")

In [23]:
def break_sentence(sentence):
    import re
    split_pattern = r'\s*(?=Arg|Return)' 
    sentences = re.split(split_pattern, sentence)
    return [s.strip() for s in sentences if s.strip()]

def get_func_description(sentence):
    return break_sentence(sentence)[0].replace("\n", "")

def get_arg_description(sentence):
    try:
        return break_sentence(sentence)[1].replace("\n", "")
    except:
        return []
    
def get_return_description(sentence):
    try:
        return break_sentence(sentence)[2].replace("\n", "")
    except:
        return []

In [27]:
def parse_args(docstring):
    try:
        pattern = r'(\w+) \((\w+)(, optional)?\): ([^.]+)(?:\. Defaults to (\w+))?'
        args_dict = {}
        matches = re.findall(pattern, docstring)

        for match in matches:
            arg_name, arg_type, optional, description, default = match
            args_dict[arg_name] = {
                "type": arg_type,
                "description": description.strip(),
                "default": default if default else (None if optional else "Required")
            }

        return args_dict
    except:
        return None

In [28]:
def format_function_signature(name, args_dict):
    try:
        signature = f"{name}("
        args = [f"{arg_name}: {arg_info['type']}" for arg_name, arg_info in args_dict.items()]
        signature += ", ".join(args)
        signature += ")"
        return signature
    except:
        return name

In [90]:
def format_arg_description(args_dict):
    try:
        formatted_string = "Args:\n\n"
        if len(args_dict)  == 0:
            return ""
        for arg_name, arg_info in args_dict.items():
            arg_type = arg_info['type']
            description = arg_info['description']
            formatted_string += f"• {arg_name} ({arg_type}): {description}\n\n"
        return formatted_string
    except:
        return ""
    
def format_return_description(args_dict):
    try:
        formatted_string = "Returns:\n\n"
        if len(args_dict)  == 0:
            return ""
        for arg_name, arg_info in args_dict.items():
            arg_type = arg_info['type']
            description = arg_info['description']
            formatted_string += f"•        {arg_name} ({arg_type}): {description}\n\n"
        return formatted_string
    except:
        return ""
    
def join(func_des, arg_des, return_des):
    return func_des + "\n\n" + arg_des + return_des

In [91]:
api_info = pd.read_csv("Extracted_Class_Functions_and_Descriptions.csv")
api_info["Function Description"] = api_info['Description'].apply(get_func_description)
api_info["Arg Description"] = api_info['Description'].apply(get_arg_description)
api_info["Return Description"] = api_info['Description'].apply(get_return_description)
api_info["Arg Description"] = api_info["Arg Description"].apply(parse_args)
api_info["Return Description"] = api_info["Return Description"].apply(parse_args)
api_info["Function Name"] = api_info.apply(lambda row: format_function_signature(row["Function Name"], row["Arg Description"]), axis=1)
api_info["Arg Description"] = api_info["Arg Description"].apply(format_arg_description)
api_info["Return Description"] = api_info["Return Description"].apply(format_return_description)
api_info["Description"] = api_info.apply(lambda row: join(row["Function Description"], row["Arg Description"], row["Return Description"]), axis=1)
updated_api_info = api_info.drop(columns=['Function Description', 'Arg Description', 'Return Description'])

In [92]:
updated_api_info

,File,Class Name,Function Name,Description
0,gorilla_file_system.py,GorillaFileSystem,pwd(),Return the current working directory path.\n\n...
1,gorilla_file_system.py,GorillaFileSystem,ls(a: bool),List the contents of the current directory.\n\...
2,gorilla_file_system.py,GorillaFileSystem,cd(folder: str),Change the current working directory to the sp...
3,gorilla_file_system.py,GorillaFileSystem,mkdir(dir_name: str),Create a new directory in the current director...
4,gorilla_file_system.py,GorillaFileSystem,touch(file_name: str),Create a new file in the current directory.\n\...
...,...,...,...,...
121,vehicle_control.py,VehicleControlAPI,"estimate_distance(cityA: str, cityB: str)",Estimates the distance between two cities.\n\n...
122,vehicle_control.py,VehicleControlAPI,get_zipcode_based_on_city(city: str),Gets the zipcode based on the city.\n\nArgs:\n...
123,vehicle_control.py,VehicleControlAPI,set_navigation(destination: str),Navigates to the destination.\n\nArgs:\n\n• de...
124,vehicle_control.py,VehicleControlAPI,check_tire_pressure(tirePressure: dict),Checks the tire pressure of the vehicle.\n\nAr...


In [93]:
updated_api_info.to_csv('api_info.csv', index=False)